In [1]:
'''
以下代码将实现以下功能

1. 自定义单支网络（具体代码参考 from models.base_model import base_model）
2. 定义Siamese双支网络欧式距离
3. 根据距离大小定义样本对是否匹配
4. 定义输入样本对生成器（具体代码实现参考 from utils.images_generator import generator）
5. 通过model.fit_generator训练模型，保存模型
'''

'\n以下代码将实现以下功能\n\n1. 自定义单支网络（具体代码参考 from models.base_model import base_model）\n2. 定义Siamese双支网络欧式距离\n3. 根据距离大小定义样本对是否匹配\n4. 定义输入样本对生成器（具体代码实现参考 from utils.images_generator import generator）\n5. 通过model.fit_generator训练模型，保存模型\n'

In [1]:
import os
import cv2
import csv
import tensorflow as tf
import numpy as np
import random
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.optimizers import SGD,RMSprop
from keras.models import Model, Sequential,load_model
from keras.layers import Input,Concatenate, Add,Subtract,Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam,RMSprop,SGD

Using TensorFlow backend.


In [2]:
#set GPU to memory-growth mode

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# calculate euclidean distance
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


# calculate loss
def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

# calculate accuracy
def compute_accuracy(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    print('pred:', pred)
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [4]:
from models.base_model import base_model

input_shape=(160,160,3)
base_network=base_model(input_shape)

base_network.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 166, 166, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        1568      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 16)        528       
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 16)        64        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)        9280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 64)        256 

In [5]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 50)           248002      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           model_1[1][0]              

In [6]:
#download training images pair
train_csv_path='/root/tf-2.0/Siamese_Face_Detection/data/FaceV5_160_train.csv'
test_csv_path='/root/tf-2.0/Siamese_Face_Detection/data/FaceV5_160_test.csv'

rows_train = csv.reader(open(train_csv_path, 'r'), delimiter=',')
imgs_train = list(rows_train)

rows_test=csv.reader(open(test_csv_path, 'r'), delimiter=',')
imgs_test = list(rows_test)

#define iterations
epochs = 5
batch_size = 20
train_iterations = len(imgs_train)//20
test_iterations = len(imgs_test)//20

#choose the proper optimizer
initial_lr=0.001
decay_rate=initial_lr/epochs
sgd=SGD(learning_rate=initial_lr,decay=decay_rate,momentum=0.9)
rms=RMSprop()
adam = Adam(0.0001)

In [7]:
from utils.images_generator import generator

model_result = '/root/tf-2.0/Siamese_Face_Detection/h5/'
#define training iterator
train_data=generator(imgs_train, batch_size,input_shape)
valid_data=validation_data = generator(imgs_test,batch_size,input_shape)

checkpoint = ModelCheckpoint(filepath=model_result + 'Siamese_softmax_{epoch:03d}.h5', verbose=1)
model.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])
history=model.fit_generator(train_data,
                            steps_per_epoch = train_iterations,
                            epochs = epochs,
                            validation_data = valid_data,
                            validation_steps = test_iterations,
                            callbacks=[checkpoint])

Epoch 1/5
3000/3000 [==============================] - 183s 61ms/step - loss: 0.0839 - accuracy: 0.9238 - val_loss: 0.0612 - val_accuracy: 0.9344

Epoch 00001: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_softmax_001.h5
Epoch 2/5
3000/3000 [==============================] - 180s 60ms/step - loss: 0.0649 - accuracy: 0.9248 - val_loss: 0.0366 - val_accuracy: 0.9445

Epoch 00002: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_softmax_002.h5
Epoch 3/5
3000/3000 [==============================] - 179s 60ms/step - loss: 0.0575 - accuracy: 0.9276 - val_loss: 0.0120 - val_accuracy: 0.9357

Epoch 00003: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_softmax_003.h5
Epoch 4/5
3000/3000 [==============================] - 180s 60ms/step - loss: 0.0504 - accuracy: 0.9320 - val_loss: 0.0330 - val_accuracy: 0.9552

Epoch 00004: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_softmax_004.h5
Epoch 5/5
3000/3000 [===========================

In [8]:
model.save('/root/tf-2.0/Siamese_Face_Detection/h5/Siamese_softmax.h5')

In [9]:
base_network.save('/root/tf-2.0/Siamese_Face_Detection/h5/Siamese_base_network.h5')